In [ ]:
import sys
sys.path.append('..')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataloader.boston_housing import BostonHousingData
from experiment_setup import get_model

%matplotlib inline
plt.style.use('seaborn')

In [ ]:
dataset = BostonHousingData()
df = dataset.df
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(16, 20))
for i, column in enumerate(df):
     plt.subplot(5, 3, i+1)
     plt.title(column)
     df[column].hist()

In [ ]:
plt.figure(figsize=(16, 20))
for i, column in enumerate(df):
     plt.subplot(5, 3, i+1)
     plt.title(column)
     plt.scatter(df[column], df['MEDV'])

In [ ]:
config = {
    'layers': [13, 256, 256, 128,1],
    'epochs': 10000,
    'patience': 10,
    'model_runs': 1
}
model_paths = [f"model/data/boston_housing_{i}.ckpt" for i in range(config['model_runs'])]

In [ ]:
i = 0
model = get_model(
    config['layers'], model_paths[i],
    (x_train, y_train), (x_val, y_val), epochs=config['epochs'],
    retrain=True, verbose=True, patience=config['patience'])

In [ ]:
# No scale best 15

predictions = model(x_val).cpu()
plt.figure(figsize=(10, 10))
plt.plot((0,50), (0, 50))
plt.scatter(predictions, y_val)